<a href="https://colab.research.google.com/github/sestys/aicrowd_captcha/blob/main/captcha_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update

In [ ]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install textdistance
!pip install pytorch
!pip install tqdm

In [ ]:
### Download data ###
!rm -rf data
!rm -f train.tar.gz test.tar.gz
!mkdir data
!wget https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/cptcha/v0.1/train.tar.gz
!wget https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/cptcha/v0.1/test.tar.gz
!wget https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/cptcha/v0.1/train_info.csv
!wget https://datasets.aicrowd.com/default/aicrowd-practice-challenges/public/cptcha/v0.1/test_info.csv
!mkdir data/train 
!mkdir data/test
!tar -C data/ -xvzf train.tar.gz
!tar -C data/ -xvzf test.tar.gz
!mv train_info.csv data/train_info.csv
!mv test_info.csv data/test_info.csv

In [14]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import string
from tqdm.notebook import tqdm
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split

# 1. Load Data

In [19]:
train_info_path = "data/train_info.csv"
test_info_path = "data/test_info.csv"

train_images_path = "data/train/"
test_images_path = "data/test/"
train_info = pd.read_csv(train_info_path)
test_info = pd.read_csv(test_info_path)

In [22]:
print('Train images:', len(glob.glob(train_images_path + '/*.png')))
print('Test images:', len(glob.glob(test_images_path + '/*.png')))


Train images: 10000
Test images: 5000


In [23]:
def plot_image(img_path):
    img = cv2.imread(img_path)
    plt.imshow(img)

In [ ]:
fig=plt.figure(figsize=(20,20))
columns = 3
rows = 3
for i in range(1, columns*rows +1):
    img = train_images_path + train_info['filename'][i]
    label = train_info['label'][i]
    fig.add_subplot(rows, columns, i)
    plot_image(img)
    print(label)
plt.show()

In [26]:
X_train, X_val= train_test_split(train_info, test_size=0.1, random_state=42)

In [33]:
labels = [x for x in train_info['label']]
labels = ''.join(labels)
letters = sorted(list(set(labels)))
len(letters) # 26 lower case + 26 upper case + 10 digits

62

In [35]:
# Character to idx mapping
vocabulary = letters + ['-']
idx2char = {k:v for k,v in enumerate(vocabulary, start=0)}
char2idx = {k:v for v,k in enumerate(vocabulary, start=0)}

# 2. Dataloaders